In [20]:
# open and standardize image files, try stuff
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.cluster import KMeans
from scipy import ndimage

dirs = ['plicatum_36','sargentii_17']

for d in dirs:
    files = []
    for file in glob.glob(d+'/*.ti*'):
        files.append(file)
    
    # open first image to get the image dimensions
    im = np.array(Image.open(files[0]))
    
    # define image_stack array
    image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
    # read and standardize all images
    for i in range(len(files)):
        im = np.array(Image.open(files[i]))        
        
        mean = np.mean(im)
        std = np.std(im)
        image_stack[i,:,:] = (im - mean)/std
        
        # for every nth images, explore
        if i%100 == 0:
            indx = files[i].index('.t')
            
            # histogram
            #hist, bin_edges = np.histogram(im,bins=1000,range=(0,50000))
            # to avoid log(0) later
            #hist[hist < 1] == 0.1
            #bins = (bin_edges[1:]+bin_edges[:-1])/2e0            
            
            #plt.close()
            #width = (bin_edges[1] - bin_edges[0])
            #center = (bin_edges[:-1] + bin_edges[1:]) / 2
            #plt.xlabel('pixel intensity')
            #plt.ylabel('log(count)')
            #plt.xlim([0,50000])
            #plt.ylim([0,5])
            #plt.bar(bin_edges[:-1], np.log10(hist), width)
            #plt.tight_layout(w_pad=0, h_pad=0)
            #plt.savefig('imgs/histo_'+d+'_'+files[i][indx-4:indx]+'.png')
            #plt.close()
            
            # LBP
            #lbp = local_binary_pattern(im, 8, 1)
            #plt.imshow(lbp)
            #plt.savefig('imgs/lbp_'+d+'_'+files[i][indx-4:indx]+'.png')
            #plt.close()            

            # k means clustering
            clustered_img = KMeans(2).fit_predict(im.reshape([np.prod(np.shape(im)),1]))            
            
            clustered_img[clustered_img == clustered_img[0]] = 2e0
            clustered_img[clustered_img != clustered_img[0]] = 3e0
            
            clustered_img = clustered_img.reshape(np.shape(im))
            plt.imshow(clustered_img)
            plt.savefig('imgs/kmeans_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()
            
            # label continouos regions
            labeled_img, num_regions = ndimage.label(clustered_img)
            
            plt.imshow(labeled_img)
            plt.savefig('imgs/labeled_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()
            
            # generate the size histogram of different regions
            histo = np.zeros(num_regions)
            for j in range(num_regions):
                histo[j] = np.sum(labeled_img == j)
            
            histo[histo == 0] = 0.1
            histo = np.sort(histo)[::-1]
            
            plt.xlabel('region label')
            plt.ylabel('region size')
            plt.xlim([0,20])
            plt.ylim([0,6])
            plt.bar(range(num_regions), np.log10(histo), np.zeros(num_regions)+1e0)
            plt.tight_layout(w_pad=0, h_pad=0)
            plt.savefig('imgs/area_histo_'+d+'_'+files[i][indx-4:indx]+'.png')
            plt.close()     


In [10]:
# pickle dump the clustered image_stack arrays so i don't need to always wait until all images are read and processes.
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from scipy.ndimage.interpolation import zoom
import pickle

#dirs = ['plicatum_36','sargentii_17']
dirs = ['sargentii_17']

for d in dirs:
    files = []
    for file in glob.glob(d+'/*.ti*'):
        files.append(file)
    
    # open first image to get the image dimensions
    im = zoom(np.array(Image.open(files[0])),0.5)
    
    # define image_stack array
    # use every second frame and downscale the image
    image_stack = np.zeros([len(files)/2+1,np.shape(im)[0],np.shape(im)[1]],dtype='int')
    # read images and cluster the pixels
    for i in range(len(files)):
        if i%100 == 0:
            print d,i
        
        # fill up downscaled array
        if i%2 == 0:
            # load image
            im = zoom(np.array(Image.open(files[i])),0.5)
            # do clustering
            clustered_img = KMeans(2).fit_predict(im.reshape([np.prod(np.shape(im)),1]))
            # make sure the background is 0, the bud is 1, clustering randomizes which one is which
            # assuming here that the corner pixel is always background
            if clustered_img[0] != 0:
                clustered_img ^= 1
            # reshape array
            clustered_img = clustered_img.reshape(np.shape(im))
            # save stack
            image_stack[i/2,:,:] = clustered_img

    # dump image_stack array
    f = open('data_files/'d+'.dat','w')
    pickle.dump(image_stack,f)
    f.close()


sargentii_17 0
sargentii_17 100
sargentii_17 200
sargentii_17 300
sargentii_17 400
sargentii_17 500
sargentii_17 600
sargentii_17 700
sargentii_17 800
sargentii_17 900


In [5]:
# try marching cubes to extract the surface of the bud
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from scipy.ndimage.interpolation import zoom
import pickle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mayavi import mlab

dirs = ['plicatum_36','sargentii_17']


for d in dirs:
    f = open('data_files/'+d+'.dat','r')
    image_stack = pickle.load(f)
    f.close()

    zoomed = zoom(image_stack,0.25)

    # march the cubes
    verts, faces = measure.marching_cubes(zoomed, 0.5)
    # GEOM view, visit (check on oscar) - for visualization
    
    # make the plot with matplotlib
    fig = plt.figure(figsize=(10, 12))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces])
    ax.add_collection3d(mesh)
    ax.set_xlabel("x-axis")
    ax.set_ylabel("y-axis")
    ax.set_zlabel("z-axis")
    ax.set_xlim(0, np.shape(zoomed)[0])  
    ax.set_ylim(0, np.shape(zoomed)[1])  
    ax.set_zlim(0, np.shape(zoomed)[2]) 
    plt.savefig('imgs/marching_cubes_'+d+'_plt.png')
    plt.close()
    
    for i in range(360):
        # make the plot with mlab
        mlab.triangular_mesh([vert[0] for vert in verts],
                  [vert[1] for vert in verts],
                  [vert[2] for vert in verts], faces) 
        mlab.view(azimuth = i)
        mlab.savefig('ani_'+d+'/ani_'+str(i).zfill(3)+'.png')
        mlab.close()
        
    # to convert the pngs into a movie use
    # ffmpeg -framerate 25 -i ani_%03d.png -s:v 640x360 -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p ani.mp4
    
    

NameError: name 'stop' is not defined

In [3]:
# for oscar: generate verts and faces and save the results in various resolutions
# do not run on laptop
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from scipy.ndimage.interpolation import zoom
import pickle

dirs = ['plicatum_36','sargentii_17']

for d in dirs:
    f = open('data_files/'+d+'.dat','r')
    image_stack = pickle.load(f)
    f.close()
    
    ## march the cubes - full resolution
    #verts, faces = measure.marching_cubes(image_stack, 0.5)
    #f = open('data_files/verts_faces_'+d+'_full_res.dat','w')
    #pickle.dump([verts, faces],f)
    #f.close()

    # march the cubes - downscale by a factor of 2
    verts, faces = measure.marching_cubes(zoom(image_stack,0.5), 0.5)
    f = open('data_files/verts_faces_'+d+'_half_res.dat','w')
    pickle.dump([verts, faces],f)
    f.close()

    ## march the cubes - downscale by a factor of 4
    #verts, faces = measure.marching_cubes(zoom(image_stack,0.25), 0.5)
    #f = open('data_files/verts_faces_'+d+'_quarter_res.dat','w')
    #pickle.dump([verts, faces],f)
    #f.close()

    ## march the cubes - downscale by a factor of 8
    #verts, faces = measure.marching_cubes(zoom(image_stack,0.125), 0.5)
    #f = open('data_files/verts_faces_'+d+'_octo_res.dat','w')
    #pickle.dump([verts, faces],f)
    #f.close()
    

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:549: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [ ]:
# work with the verts, faces
# separate the unconnected surfaces, and clean the data by removing the small isolated surfaces (noise).
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from scipy.ndimage.interpolation import zoom
import pickle
from mayavi import mlab


dirs = ['sargentii_17','plicatum_36']

res = 'half'

def surface_finder(faces,surface,elements_removed):
    unique_elements = np.unique(surface)
    elements_to_check = np.setdiff1d(unique_elements,elements_removed,assume_unique=True)
    if np.all([np.any(e not in faces) for e in elements_to_check]):
        return faces, surface, elements_removed
    else:
        indcs = np.any([np.any(faces == e,axis=1) for e in elements_to_check],axis=0)
        surf = faces[indcs,:]
        new_surface = np.concatenate((surface,surf),axis=0)
        new_faces = faces[~indcs,:]
        elements_removed = np.concatenate((elements_removed,elements_to_check))
        return surface_finder(new_faces,new_surface,elements_removed)

for d in dirs:

    f = open('data_files/verts_faces_'+d+'_'+res+'_res.dat','r')
    [verts, faces] = pickle.load(f)
    f.close()

    # verts are a list of 3D point coordinates.
    # faces describe which 3 points form a triangle that should be shaded.
    # e.g., this is one triangle:
    # verts = [[  1.    3.   32.5]
    # [  0.5   3.   33. ]
    # [  1.    2.5  33. ]]
    # faces = [[ 0  1  2]]
    # point 0 is [  1.    3.   32.5]
    # point 1 is [  0.5   3.   33. ]
    # point 2 is [  1.    2.5  33. ]
    # faces [ 0  1  2] describes that these three points form a triangle.
    #
    # separate surfaces do not have common points.
    
    print np.shape(verts)
    print np.shape(faces)
    
    # find separate surfaces
    surfaces = []
    
    while np.shape(faces)[0] > 0:
        print 'new surface',np.shape(faces)[0]
        surface = [faces[0,:]]
        faces = np.delete(faces,0,axis=0)    
        elements_removed = np.empty(0,dtype=int)

        faces, surface, elements_removed = surface_finder(faces,surface,elements_removed)
        unique_elements = np.unique(surface)
        if np.any([np.any(e in faces) for e in unique_elements]):
            print 'there are points left in faces that should be in surfaces'
            print unique_elements
            print np.shape(surface),np.shape(faces)
            raise ValueError
        surfaces.append(surface)
    
    f = open('data_files/surfaces_'+d+'_'+res+'_res.dat','w')
    pickle.dump(surfaces,f)
    f.close()
    
    surf_size = np.zeros(len(surfaces))
    for i in range(len(surfaces)):
        surf_size[i] = np.shape(surfaces[i])[0]
    
    print surf_size
    stop
    
    

In [2]:
# make the movie of various surfaces
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from scipy.ndimage.interpolation import zoom
import pickle
from mayavi import mlab

dirs = ['sargentii_17','plicatum_36',]

for d in dirs:

    f = open('data_files/verts_faces_'+d+'_'+res+'_res.dat','r')
    [verts, faces] = pickle.load(f)
    f.close()
    
    f = open('data_files/surfaces_'+d+'_'+res+'_res.dat','r')
    surfaces = pickle.load(f)
    f.close()
    
    surf_size = np.zeros(len(surfaces))
    for i in range(len(surfaces)):
        surf_size[i] = np.shape(surfaces[i])[0]
    
    
    sorted_indcs = np.argsort(surf_size)
        
    # make the plot with mlab
    for i in range(9):
        mlab.triangular_mesh([vert[0] for vert in verts],
              [vert[1] for vert in verts],
              [vert[2] for vert in verts], surfaces[sorted_indcs[-1]]) 
        mlab.view(azimuth = i*40)
        mlab.savefig('ani_'+d+'/ani_'+str(i).zfill(3)+'.png')
        mlab.close()
    

(486378, 3)
(664342, 3)
